In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from datetime import datetime

# define the ticker symbols and the start and end dates
tickers = ["BRIS.JK", "BBCA.JK"]
start_date = dt.datetime.now() - dt.timedelta(days=365*5)
end_date = dt.datetime.now()

# download the historical prices for the tickers from Yahoo Finance and store in a dictionary
prices_dict = {}
dfs = []
dfs_pred = []
for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date)
    df["Ticker"] = ticker
    dfs.append(df)
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size, :]
    X_train = []
    y_train = []
    for i in range(360, len(train_data)):
        X_train.append(train_data[i-360:i, 0])
        y_train.append(train_data[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=1, epochs=10)
    test_data = data[train_size-360:, :]
    X_test = []
    y_test = data[train_size:, :]
    for i in range(360, len(test_data)):
        X_test.append(test_data[i-360:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    dates = pd.date_range(start=end_date, periods=len(predictions)+1, freq='D')[1:]
    df_pred = pd.DataFrame({'Date': dates, 'Close': predictions.flatten()}, index=dates)
    df_pred['Ticker'] = ticker
    dfs_pred.append(df_pred)
    prices_dict[ticker] = pd.concat([df, df_pred], axis=0)

# combine the dataframes into one
df_all = pd.concat(prices_dict.values(), keys=prices_dict.keys(), names=['Ticker', 'Date'])

# plot the actual and predicted prices for each ticker
import matplotlib.pyplot as plt
plt.figure(figsize=(16,8))
plt.title('Stock Price Prediction')
for ticker in tickers:
    plt.plot(df_all.loc[ticker]['Close'], label=ticker)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.legend()
plt.show()

In [ ]:
# plotting Actual vs Predicted Stock Price
df = pd.concat([df for df in dfs], ignore_index=False)
df_pred = pd.concat([df_pred for df_pred in dfs_pred], ignore_index=False)
plt.figure(figsize=(16,8))
for ticker in tickers:
    a = ticker + ' Actual'
    df_ticker = df[df['Ticker'] == ticker]
    plt.plot(df_ticker['Close'], label=a)

for ticker in tickers:
    a = ticker + ' Predicted'
    df_ticker = df_pred[df_pred['Ticker'] == ticker]
    plt.plot(df_ticker['Close'], label=a)

plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.legend()
plt.show()

In [ ]:
# see when the max predicted price is 
m = []
for ticker in tickers:
    df_ticker = df_pred[df_pred['Ticker'] == ticker]
    m.append(max(df_ticker["Close"]))

df_all[df_all['Close'].isin(m)]